# Домашнее задание 3

В этом задании напишем простое решение классификации датасета `FashionMNIST`, а затем будем его улучшать с помощью:
- dropout;
- batch normalization;
- LR scheduler;

В конце сохраним модель в файл и убедимся, что этот файл можем в дальнейшем прочитать.

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision.datasets import FashionMNIST
from torchvision.transforms import ToTensor
from dataclasses import dataclass

In [2]:
train_dataset = FashionMNIST(
    root="./data", train=True, download=True, transform=ToTensor()
)
test_dataset = FashionMNIST(
    root="./data", train=False, download=True, transform=ToTensor()
)

100%|██████████| 26421880/26421880 [00:06<00:00, 4142799.43it/s]


Extracting ./data\FashionMNIST\raw\train-images-idx3-ubyte.gz to ./data\FashionMNIST\raw



100%|██████████| 29515/29515 [00:00<00:00, 539674.62it/s]


Extracting ./data\FashionMNIST\raw\train-labels-idx1-ubyte.gz to ./data\FashionMNIST\raw



100%|██████████| 4422102/4422102 [00:00<00:00, 6721111.62it/s]


Extracting ./data\FashionMNIST\raw\t10k-images-idx3-ubyte.gz to ./data\FashionMNIST\raw



100%|██████████| 5148/5148 [00:00<?, ?it/s]

Extracting ./data\FashionMNIST\raw\t10k-labels-idx1-ubyte.gz to ./data\FashionMNIST\raw



In [3]:
X_train = train_dataset.data.float()
y_train = train_dataset.targets
X_test = test_dataset.data.float()
y_test = test_dataset.targets

In [4]:
@dataclass
class TrainConfig:
    lr: float = 1e-3
    total_iterations: int = 100


# Для оценки будем использовать метрику accuracy
# Подумайте (опционально), какие еще метрики можно использовать
def calculate_accuracy(y_pred: torch.Tensor, y_true: torch.Tensor) -> float:
    _, predicted = torch.max(y_pred, 1)
    correct = (predicted == y_true).float().sum()
    accuracy = correct / y_true.shape[0]
    return accuracy.item()

## Задание №1

Попробуйте реализовать простой бейзлайн с несколькими слоями:
- Linear
- ReLU
- Linear

Вставьте свою релизацию `SimpleModel` в проверку.
Вам нужно дописать и сдать как `SimpleModel`, так и `train_loop`.

Используйте кросс-энтропию как функцию потерь.

In [6]:
# Возможно, класс нужно отнаследовать от некого класса из pytorch
class SimpleModel:
    # размерность после первого слоя
    hidden_dim = 512


def train_loop(
    model: SimpleModel,
    X_train: torch.Tensor,
    y_train: torch.Tensor,
    X_val: torch.Tensor,
    y_val: torch.Tensor,
    config: TrainConfig,
):
    """Обучите здесь модель, подсчитайте метрики на валидационной выборке.

    Можете так же писать/рисовать accuracy в процессе обучения.
    Например, каждые 10 итераций или даже каждую итерацию.
    """
    # Оставьте такое название перменной, это требование проверяющей системы
    optimizer = optim.SGD(...)
    # Ваш код обучения модели
    ...

## Задание №2
Какое максимальное значение метрики accuracy удалось получить в процессе обучения?

Округлите до 3 значений после запятой


In [7]:
torch.manual_seed(987)
# дефолтные значения
config = TrainConfig()
# Ваш код для обучения и подсчета accuracy

## Задание №3
Добавьте один `dropout` слой в вашу модель.

_Подумайте, что может поменяться при перестановке ReLU и Dropout слоев местами._

In [8]:
# Возможно, класс нужно отнаследовать от некого класса из pytorch
class DropoutModel:
    hidden_dim = 512
    ...

In [9]:
torch.manual_seed(987)
config = TrainConfig()
# Ваш код для обучения и подсчета accuracy

## Задание №4
Какое максимальное значение accuracy получилось в ходе обучения модели? 

Округлите до 3х знаков после запятой и отправьте в ЛМС.

## Задание №5

Добавьте `BatchNorm` в вашу модель.
Отправьте в ЛМС реализацию.

Стоит ли делать BatchNorm до ReLU или после него?
Это дискуссионный вопрос, чаще всего применяют сначала нелинейность, затем Batch Norm.
Один из аргументов: при таком подходе данные на выходе будут иметь среднее 0 - что и ожидают люди, когда добавляют нормализацию.

_[Дискуссия на Reddit](https://www.reddit.com/r/MachineLearning/comments/67gonq/d_batch_normalization_before_or_after_relu/)_

Для определенности в этом задании будем следовать такому порядку: сначала ReLU, затем Batch Norm.

In [10]:
# Возможно, класс нужно отнаследовать от некого класса из pytorch
class BatchNormModel:
    hidden_dim = 512
    ...

In [11]:
torch.manual_seed(987)
config = TrainConfig()
# Ваш код для обучения и подсчета accuracy

## Задание №6
Какое максимальное значение `accuracy` получилось в ходе обучения модели? 

Округлите до 3х знаков после запятой.

Результат batch normalization мог не особо порадовать.
Но не спешите с выводами насчет этого слоя!

Попробуйте обучить заново все три модели со значением `lr=1e-2` (в 10 раз больше).
Сравните результаты моделей и сделайте вывод.

## Задание №7
Добавьте `LRscheduler` в вашу модель.

Подробнее про `schedulers` можно почитать в [документации](https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate)

In [13]:
from torch.optim.lr_scheduler import StepLR


def train_loop_with_scheduler(
    model,
    X_train: torch.Tensor,
    y_train: torch.Tensor,
    X_val: torch.Tensor,
    y_val: torch.Tensor,
    config: TrainConfig,
):
    ...
    scheduler = StepLR(..., step_size=5, gamma=0.1)
    ...

In [14]:
torch.manual_seed(987)
config = TrainConfig(lr=1e-3)
# Ваш код для обучения и подсчета accuracy

## Задание №8

Поэксперементируйте с параметрами нейронной сети, попробуйте добиться максимальной метрики `accuracy`.

- попробуйте комбинацию Drouput + Batch Normalization и подумайте, как лучше всего раскрыть силу batch normalization (вспомните эксперименты с lr);
- попробуйте подвигать вероятность в Dropout;
- ну, или подержите обучение подольше, поставив больше шагов :)

В ЛМС нужно сдать код класса `ExpModel`.
Вам необходимо выбить accuracy > 80%, чтобы сдать этот пункт.

In [27]:
torch.manual_seed(987)


# Ваш код модели и ее обучения при seed = 987
class ExpModel: ...


model = ExpModel()
config = TrainConfig(...)
train_loop(model, X_train, y_train, X_test, y_test, config)

Наконец, сохраним лучшую модель, чтобы в будущем ее могли взять и использовать, без обучения.

## Задание №9

Напишите код, который сохранит модель в файл `model.pt`.

In [29]:
# Впоследствии эту модель можно будет загрузить вот так:
model_loaded = ExpModel(num_classes=len(y_test.unique()))
model_loaded.load_state_dict(torch.load("model.pt"))